In [1]:
# Python Map Visualisation of Covid-19
import json
import folium 
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

In [2]:
conn = http.client.HTTPSConnection("api.covid19api.com")
payload=''
headers = {}
conn.request("GET", "/summary", payload, headers)
res = conn.getresponse()
data = res.read().decode('UTF-8')

In [4]:
covid1 = json.loads(data)

In [5]:
#normalise data
pd.json_normalize(covid1['Countries'], sep=',');

In [6]:
#convert to pandas dataframe

df = pd.DataFrame(covid1['Countries'])
#df

In [7]:
#drop unwanted cols
covid = df.drop(columns=['CountryCode', 'Slug', 'Date', 'Premium'], axis=1)
#covid

In [8]:
map1 = folium.Map(tiles="Stamen Terrain", min_zoom=1.5)
#map1

In [9]:
url = 'http://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

In [10]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='Covid-19',
    data=covid,
    columns=['Country', 'TotalConfirmed'],
    key_on = 'feature.properties.name',
    fill_color='YlOrRd',
    nana_fill_color='black',
    legend_name='Total Confirmed Covid-19 Cases',
).add_to(map1)
#map1

&lt;folium.features.Choropleth at 0x2a22164d340&gt;

In [11]:
covid.update(covid['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid.update(covid['TotalRecovered'].map('Total Recovered:{}'.format))

In [12]:
coordinates=pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
#coordinates

In [13]:
covid_final=pd.merge(covid, coordinates, on='Country')

def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
    radius=5,
    weight=2,
    popup = [point.Country, point.TotalConfirmed, point.TotalRecovered],
    fill_color = '#000000').add_to(map1)

In [14]:
# 1. Total Confirmed Covid-19 Cases

covid_final.apply(plotDot, axis=1)
map1.fit_bounds(map1.get_bounds())
map1
#map1.save('cases.html')


&lt;folium.folium.Map at 0x2a2215bc730&gt;

In [15]:
map1 = folium.Map(tiles='StamenToner', min_zoom=2)
#map

In [16]:
deaths = covid_final['TotalDeaths'].astype(float)
lat = covid_final['latitude'].astype(float)
lon = covid_final['longitude'].astype(float)
map1.add_child(HeatMap(zip(lat,lon,deaths), radius=0)); 

In [17]:
# 2. Total Confirmed Covid-19 Deaths
def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
    radiu=2,
    weight=1,
    popup=[point.Country, point.TotalDeaths],
    fill_color='#000000').add_to(map1)
covid_final.apply(plotDot, axis=1)
map1.fit_bounds(map1.get_bounds())
map1
#map1.save('deaths.html')

&lt;folium.folium.Map at 0x2a221a61460&gt;